In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, LSTM, Embedding
from tensorflow.keras.models import Model

In [ ]:
df = pd.read_csv('data.csv')
df = df.sample(frac=1)

In [ ]:
X = df.tweet
Y = df["class"].values
df_train, df_test, y_train, y_test = train_test_split(X,Y,test_size=0.20)

In [ ]:
MAX_VOCAB_SIZE = 20000
tokenizer = Tokenizer(num_words = MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(df_train)
sequences_train = tokenizer.texts_to_sequences(df_train)
sequences_test = tokenizer.texts_to_sequences(df_test)

In [ ]:
V = len(tokenizer.word_index)
print(V)

10822


In [ ]:
data_train = pad_sequences(sequences_train)
print(data_train.shape)

(2560, 75)


In [ ]:
T = data_train.shape[1]

In [ ]:
data_test = pad_sequences(sequences_test,maxlen=T)

In [ ]:
data_test.shape

(640, 75)

In [ ]:
from tensorflow.keras import initializers
D = 10 #embedding dimensionality 

M = 5 #hidden state vector


i = Input(shape=(T,))
x = Embedding(V+1,D)(i)
x = LSTM(M,return_sequences=True,kernel_initializer=initializers.RandomNormal(stddev=0.01))(x)
x = GlobalMaxPooling1D()(x)
x = Dense(1,activation="sigmoid")(x)

model = Model(i,x)

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(data_train,y_train,epochs=10,validation_data=(data_test,y_test))

Epoch 1/10
80/80 [==============================] - 4s 28ms/step - loss: 0.6924 - accuracy: 0.5234 - val_loss: 0.6885 - val_accuracy: 0.4859
Epoch 2/10
80/80 [==============================] - 2s 22ms/step - loss: 0.6438 - accuracy: 0.6246 - val_loss: 0.5713 - val_accuracy: 0.8594
Epoch 3/10
80/80 [==============================] - 2s 22ms/step - loss: 0.4109 - accuracy: 0.9160 - val_loss: 0.4211 - val_accuracy: 0.8672
Epoch 4/10
80/80 [==============================] - 2s 22ms/step - loss: 0.2509 - accuracy: 0.9703 - val_loss: 0.3412 - val_accuracy: 0.9094
Epoch 5/10
80/80 [==============================] - 2s 22ms/step - loss: 0.1838 - accuracy: 0.9863 - val_loss: 0.3339 - val_accuracy: 0.9078
Epoch 6/10
80/80 [==============================] - 2s 22ms/step - loss: 0.1436 - accuracy: 0.9906 - val_loss: 0.3113 - val_accuracy: 0.9109
Epoch 7/10
80/80 [==============================] - 2s 22ms/step - loss: 0.1142 - accuracy: 0.9961 - val_loss: 0.3235 - val_accuracy: 0.9062
Epoch 8/10
80

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
embed_dim = 10  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 15  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(T,))
x = Embedding(V+1,D)(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1,)(x)
x = layers.Dense(20, activation="elu",kernel_initializer=initializers.RandomNormal(stddev=0.01))(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
history = model.fit(data_train, y_train, epochs=10, validation_data=(data_test, y_test))

Epoch 1/10
80/80 [==============================] - 3s 23ms/step - loss: 0.6935 - accuracy: 0.5012 - val_loss: 0.6899 - val_accuracy: 0.5969
Epoch 2/10
80/80 [==============================] - 2s 20ms/step - loss: 0.6205 - accuracy: 0.6508 - val_loss: 0.5359 - val_accuracy: 0.7031
Epoch 3/10
80/80 [==============================] - 2s 20ms/step - loss: 0.1786 - accuracy: 0.9363 - val_loss: 0.2629 - val_accuracy: 0.9094
Epoch 4/10
80/80 [==============================] - 2s 19ms/step - loss: 0.0752 - accuracy: 0.9750 - val_loss: 0.4031 - val_accuracy: 0.8578
Epoch 5/10
80/80 [==============================] - 2s 20ms/step - loss: 0.0306 - accuracy: 0.9910 - val_loss: 0.3072 - val_accuracy: 0.9156
Epoch 6/10
80/80 [==============================] - 2s 20ms/step - loss: 0.0186 - accuracy: 0.9945 - val_loss: 0.3013 - val_accuracy: 0.9141
Epoch 7/10
80/80 [==============================] - 2s 20ms/step - loss: 0.0115 - accuracy: 0.9977 - val_loss: 0.4889 - val_accuracy: 0.8734
Epoch 8/10
80

In [ ]:
import joblib

In [ ]:
joblib.dump(model,'blah')
joblib.dump(tokenizer,'tok')

INFO:tensorflow:Assets written to: ram://363b37d5-d9ff-4a73-be11-156c072a61fb/assets


INFO:tensorflow:Assets written to: ram://363b37d5-d9ff-4a73-be11-156c072a61fb/assets


['tok']

In [ ]:
!pwd

/content


In [ ]:
l = ['this is hate speech']
l = tokenizer.texts_to_sequences(l)
l = pad_sequences(l, maxlen=75)

In [ ]:
model.predict(l)

array([[0.99450743]], dtype=float32)